In [ ]:
import os
import numpy as np
import tensorflow as tf

In [ ]:
# change paths and copy data if on google colab
is_colab = False

datapath = 'training_data.npz'
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    datapath = '/content/gdrive/MyDrive/' + datapath
    is_colab = True
except:
    print('Not on Google Colab!')

data = np.load(datapath)
X_data = data['X']
Y_data = data['Y']

In [ ]:
length = X_data.shape[0]
test_length = int(length*0.1)
indices = np.arange(start=0, stop=length)
np.random.shuffle(indices)
Y_train = np.take(Y_data, indices[:-test_length], axis=0)
Y_test = np.take(Y_data, indices[-test_length:], axis=0)
X_train = np.take(X_data, indices[:-test_length], axis=0)
X_test = np.take(X_data, indices[-test_length:], axis=0)

In [ ]:
import tensorflow as tf
from tensorflow.python import keras as keras
from keras import activations
from keras import models
from keras import layers
from keras.applications.inception_v3 import InceptionV3

!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
inception = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights=None)
inception.load_weights(local_weights_file)
for layer in inception.layers:
  layer.trainable = False

inner_model = models.Functional(inputs=inception.layers[1].input, outputs=inception.layers[288].output)

model = models.Sequential()
model.add(layers.Rescaling(scale=2./255, offset=-1, input_shape=(150, 150, 3)))
model.add(layers.RandomFlip(mode='horizontal'))
model.add(layers.RandomRotation(factor=0.06, fill_mode='reflect', interpolation='bilinear'))
model.add(layers.RandomTranslation(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1), fill_mode='reflect', interpolation='bilinear'))
model.add(layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1), fill_mode='reflect', interpolation='bilinear'))
model.add(inner_model)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(64, activation='sigmoid'))
model.add(layers.Dense(17, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

history = model.fit(X_train, Y_train, epochs=30, batch_size=32, validation_data=(X_test, Y_test))
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Rescaling(scale=1./255, input_shape=(150, 150, 3)))
model.add(layers.RandomFlip(mode='horizontal'))
model.add(layers.RandomRotation(factor=0.06, fill_mode='reflect', interpolation='bilinear'))
model.add(layers.RandomTranslation(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1), fill_mode='reflect', interpolation='bilinear'))
model.add(layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1), fill_mode='reflect', interpolation='bilinear'))
model.add(layers.convolutional.Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(150,150,3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.convolutional.Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.convolutional.Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.convolutional.Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(17, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, Y_train, epochs=30, batch_size=32, validation_data=(X_test, Y_test))
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()